# Secrets in Key Vault

You can use `DefaultAzureCredential` to easily authenticate in both development and production environments without changing code. This credential type detects common production environment configurations and, failing to find any, detects common developer credentials like those from the [Azure CLI](https://aka.ms/azure-cli) or [Visual Studio](https://www.visualstudio.com).

## Creating the SecretClient

To create a `SecretClient`, you'll need to know the URI to your Key Vault. This may appear as "DNS Name" in the [Azure Portal](https://portal.azure.com/#blade/HubsExtension/BrowseResource/resourceType/Microsoft.KeyVault%2Fvaults). Replace "heathskv2.vault.azure.net" with your vault URI.

In [1]:
#r "nuget:Azure.Identity"
#r "nuget:Azure.Security.KeyVault.Secrets"

#pragma warning disable CS1701

using Azure.Identity;
using Azure.Security.KeyVault.Secrets;

Uri vaultUri = new Uri("https://heathskv2.vault.azure.net");
DefaultAzureCredential credential = new DefaultAzureCredential();
SecretClient client = new SecretClient(vaultUri, credential);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Azure.Security.KeyVault.Secrets version 4.1.0

Installed package Azure.Identity version 1.2.2

You'll need to authenticate. You can do this easily using the CLI:

In [2]:
#!pwsh
az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FYY85TLJH to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "96be4b7a-defb-4dc2-a31f-49ee6145d5ab",
    "id": "c649122d-c5c8-40b0-b95c-e09da8dbfdf0",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Windows Azure MSDN - Visual Studio Ultimate",
    "state": "Enabled",
    "tenantId": "96be4b7a-defb-4dc2-a31f-49ee6145d5ab",
    "user": {
      "name": "heaths@outlook.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "96be4b7a-defb-4dc2-a31f-49ee6145d5ab",
    "id": "22f692c2-e192-4019-b0f8-884cefc0f1a1",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Visual Studio Enterprise",
    "state": "Enabled",
    "tenantId": "96be4b7a-defb-4dc2-a31f-49ee6145d5ab",
    "user": {
      "name": "heaths@outlook.com",
      "type": "user"
    }
  }
]


## Creating a secret

To create a secret, you need to specify a name and value:

In [3]:
#pragma warning disable CS1701

await client.SetSecretAsync("secret-name", "secret-value");

## Retrieving secrets

When you enumerate secrets, the secret value is not retrieved. This is because the "list" permission for enumerating secrets can be assigned separately from the "get" permission for getting secrets. Instead, a `SecretProperties` object is returned which provides the name and other information.

Given the name, you can retrieve the secret value:

In [4]:
#pragma warning disable CS1701

await foreach (SecretProperties properties in client.GetPropertiesOfSecretsAsync())
{
    KeyVaultSecret secret = await client.GetSecretAsync(properties.Name);
    Console.WriteLine($"Secret: {secret.Value}, version: {secret.Properties.Version}");
}

Secret: secret-value, version: 3a27b50ad13d4933b87706a901966424


## Updating a secret

Updating a secret actually creates a new version. Secrets are immutable, and you can retrieve past version of them.

In [5]:
#pragma warning disable CS1701

KeyVaultSecret secret = await client.SetSecretAsync("secret-name", "new secret value");
Console.WriteLine($"Secret: {secret.Value}, version: {secret.Properties.Version}");

Secret: new secret value, version: 0b359bdee56c4ea78bb90d5e2c800b3d


## Deleting secrets

Deleting a secret is straight forward, but since it can take time you must wait for the operation to complete.

In [6]:
#pragma warning disable CS1701

DeleteSecretOperation operation = await client.StartDeleteSecretAsync("secret-name");
await operation.WaitForCompletionAsync();


warning CS1701: Assuming assembly reference 'Azure.Core, Version=1.0.2.0, Culture=neutral, PublicKeyToken=92742159e12e44c8' used by 'Azure.Security.KeyVault.Secrets' matches identity 'Azure.Core, Version=1.4.1.0, Culture=neutral, PublicKeyToken=92742159e12e44c8' of 'Azure.Core', you may need to supply runtime policy



If soft delete is enabled for your Key Vault - it is enabled by default now - the secret is not completely gone until it's purged:

In [7]:
#pragma warning disable CS1701

await client.PurgeDeletedSecretAsync("secret-name");

## Links

* [About Azure Key Vault secrets](https://docs.microsoft.com/azure/key-vault/secrets/about-secrets)
* [Azure Key Vault secret client library for .NET](https://docs.microsoft.com/dotnet/api/overview/azure/security.keyvault.secrets-readme?view=azure-dotnet)